In [2]:
import numpy as np
import pandas as pd
import time 
import json
import requests 
import re

In [4]:
# lets start with the attractions
attractions_df = pd.read_csv('attractions_api1000.csv')
attractions_df.head(2)

,location_id,name,distance,bearing,address_obj.street1,address_obj.city,address_obj.country,address_obj.postalcode,address_obj.address_string,address_obj.street2,address_obj.state
0,23654441.0,Glam Caffe,0.415274,south,Rua Americo Diniz 111 Casas Novas,Coimbra,Portugal,3045-138,"Rua Americo Diniz 111 Casas Novas, Coimbra 304...",NaN,NaN
1,13124988.0,Just Be,0.328461,east,Avenida Sao Sebastiao 9,Evora,Portugal,7000-767,"Avenida Sao Sebastiao 9, Evora 7000-767 Portugal",NaN,NaN


In [6]:
#attractions_df.info()
len(attractions_df)

1000

In [7]:
attractions_df.isnull().sum()

location_id                    39
name                           39
distance                       39
bearing                        39
address_obj.street1            94
address_obj.city               39
address_obj.country            39
address_obj.postalcode        113
address_obj.address_string     39
address_obj.street2           839
address_obj.state             992
dtype: int64

In [14]:
attractions_df = attractions_df.drop('address_obj.street2', axis=1)

In [15]:
attractions_df = attractions_df.drop('address_obj.state', axis=1)

In [18]:
attractions_df['address_obj.postalcode'] = attractions_df['address_obj.postalcode'].fillna(0)

In [20]:
attractions_df.isnull().sum()
# we will keep the nan values in the postal code column as we need the column for later, lets see if we can restore them somehow 

location_id                   39
name                          39
distance                      39
bearing                       39
address_obj.street1           94
address_obj.city              39
address_obj.country           39
address_obj.postalcode         0
address_obj.address_string    39
dtype: int64

In [21]:
attractions_df = attractions_df.dropna()

In [24]:
#attractions_df.isnull().sum()
attractions_df.duplicated().sum()


0

In [32]:
#attractions_df["name"].value_counts()
#attractions_df["address_obj.street1"].value_counts()

attractions_df = attractions_df.rename(columns={'address_obj.street1': 'street', 'address_obj.city': 'city', 'address_obj.country': 'country', 'address_obj.postalcode': 'postal_code', 'address_obj.address_string': 'complete_address'})

In [35]:
attractions_df['type'] = 'attraction'

In [39]:
attractions_df.head()
# attractions df is ready

,location_id,name,distance,bearing,street,city,country,postal_code,complete_address,type
0,23654441.0,Glam Caffe,0.415274,south,Rua Americo Diniz 111 Casas Novas,Coimbra,Portugal,3045-138,"Rua Americo Diniz 111 Casas Novas, Coimbra 304...",attraction
1,13124988.0,Just Be,0.328461,east,Avenida Sao Sebastiao 9,Evora,Portugal,7000-767,"Avenida Sao Sebastiao 9, Evora 7000-767 Portugal",attraction
2,7224821.0,Igreja de Nossa Senhora da Lapa,0.057385,north,Campo da Restauracao,Vila Vicosa,Portugal,7150-000,"Campo da Restauracao, Vila Vicosa 7150-000 Por...",attraction
3,7142467.0,Oficina da Formiga Ceramica,0.385437,southwest,Rua da Coutada 77,Ilhavo,Portugal,3830-134,"Rua da Coutada 77, Ilhavo 3830-134 Portugal",attraction
4,20999967.0,Memories Night Club,1.037043,north,Travessa do Monte do Paco 56,Esgueira,Portugal,3800-036,"Travessa do Monte do Paco 56, Esgueira 3800-03...",attraction


In [45]:
hotels_df = pd.read_csv('hotels_api1000.csv')
hotels_df.head()

,location_id,name,distance,bearing,address_obj.street1,address_obj.street2,address_obj.city,address_obj.country,address_obj.postalcode,address_obj.address_string
0,261349.0,The Noble House,0.107644,northwest,Rua da Freiria de Baixo 16,NaN,Evora,Portugal,7000-898,"Rua da Freiria de Baixo 16, Evora 7000-898 Por..."
1,12579601.0,The Noble House,0.104778,northwest,Rua da Freiria de Baixo 16,NaN,Evora,Portugal,7000-898,"Rua da Freiria de Baixo 16, Evora 7000-898 Por..."
2,23850713.0,Mourasuites Hotel,0.096517,southwest,Largo da Portas de Moura 28,NaN,Evora,Portugal,7000-647,"Largo da Portas de Moura 28, Evora 7000-647 Po..."
3,23710954.0,Casa Soure Suites and apartements,0.133332,west,Rua da Misericordia n0 9,NaN,Evora,Portugal,7000-654,"Rua da Misericordia n0 9, Evora 7000-654 Portugal"
4,253484.0,Hotel Riviera,0.223739,west,Rua 5 de Outubro 47 49,NaN,Evora,Portugal,7000-854,"Rua 5 de Outubro 47 49, Evora 7000-854 Portugal"


In [46]:
len(hotels_df)

5391

In [47]:
hotels_df.isnull().sum()

location_id                    260
name                           260
distance                       260
bearing                        260
address_obj.street1            268
address_obj.street2           4841
address_obj.city               260
address_obj.country            260
address_obj.postalcode         292
address_obj.address_string     260
dtype: int64

In [49]:
hotels_df = hotels_df.drop('address_obj.street2', axis=1)

In [51]:
hotels_df = hotels_df.dropna()

In [54]:
hotels_df.duplicated().sum()


11

In [55]:
hotels_df.drop_duplicates(inplace=True)

In [58]:
hotels_df = hotels_df.rename(columns={'address_obj.street1': 'street', 'address_obj.city': 'city', 'address_obj.country': 'country', 'address_obj.postalcode': 'postal_code', 'address_obj.address_string': 'complete_address'})

In [60]:
hotels_df['type'] = 'hotel'

In [61]:
hotels_df.head()
# hotel df ready but first 2 lines are equal but the df doesn´t show them as equal, still figuring out 

,location_id,name,distance,bearing,street,city,country,postal_code,complete_address,type
0,261349.0,The Noble House,0.107644,northwest,Rua da Freiria de Baixo 16,Evora,Portugal,7000-898,"Rua da Freiria de Baixo 16, Evora 7000-898 Por...",hotel
1,12579601.0,The Noble House,0.104778,northwest,Rua da Freiria de Baixo 16,Evora,Portugal,7000-898,"Rua da Freiria de Baixo 16, Evora 7000-898 Por...",hotel
2,23850713.0,Mourasuites Hotel,0.096517,southwest,Largo da Portas de Moura 28,Evora,Portugal,7000-647,"Largo da Portas de Moura 28, Evora 7000-647 Po...",hotel
3,23710954.0,Casa Soure Suites and apartements,0.133332,west,Rua da Misericordia n0 9,Evora,Portugal,7000-654,"Rua da Misericordia n0 9, Evora 7000-654 Portugal",hotel
4,253484.0,Hotel Riviera,0.223739,west,Rua 5 de Outubro 47 49,Evora,Portugal,7000-854,"Rua 5 de Outubro 47 49, Evora 7000-854 Portugal",hotel


In [66]:
# restaurants
restaurants_df = pd.read_csv('restaurants_api1000.csv')
restaurants_df.head(2)

,location_id,name,distance,bearing,address_obj.street1,address_obj.street2,address_obj.city,address_obj.country,address_obj.address_string,address_obj.postalcode
0,3941742.0,Rest. Pizzaria A Variante,1.136036,northwest,En 234-1,"40° 14' 51.19, -8° 29' 39.20",Coimbra,Portugal,"En 234-1 40° 14' 51.19, -8° 29' 39.20, Coimbra...",NaN
1,25131407.0,Alto do Padrao Restaurante,0.087935,east,Rua Senhora dos Remedios,Padrão Industrial Zone,Lousa,Portugal,Rua Senhora dos Remedios Padrão Industrial Zon...,3200-193


In [67]:
len(restaurants_df)

8979

In [68]:
restaurants_df.isnull().sum()

location_id                     13
name                            13
distance                        13
bearing                         13
address_obj.street1            148
address_obj.street2           6251
address_obj.city                13
address_obj.country             13
address_obj.address_string      13
address_obj.postalcode         375
dtype: int64

In [69]:
restaurants_df = restaurants_df.drop('address_obj.street2', axis=1)

In [70]:
restaurants_df = restaurants_df.dropna()

In [73]:
#restaurants_df.isnull().sum()
hotels_df.duplicated().sum()


0

In [74]:
restaurants_df = restaurants_df.rename(columns={'address_obj.street1': 'street', 'address_obj.city': 'city', 'address_obj.country': 'country', 'address_obj.postalcode': 'postal_code', 'address_obj.address_string': 'complete_address'})

In [75]:
restaurants_df['type'] = 'restaurant'

In [78]:
restaurants_df.head(2)
# restaurant df clean

,location_id,name,distance,bearing,street,city,country,complete_address,postal_code,type
1,25131407.0,Alto do Padrao Restaurante,0.087935,east,Rua Senhora dos Remedios,Lousa,Portugal,Rua Senhora dos Remedios Padrão Industrial Zon...,3200-193,restaurant
2,6437542.0,Restaurante Fornos de Sao Domingos,0.543903,north,"Rua do Apeadeiro, Fornos",Coimbra,Portugal,"Rua do Apeadeiro, Fornos, Coimbra 3020-927 Por...",3020-927,restaurant


In [81]:
portugal_dates_complete = pd.concat([restaurants_df, attractions_df, hotels_df])

In [82]:
portugal_dates_complete

,location_id,name,distance,bearing,street,city,country,complete_address,postal_code,type
1,25131407.0,Alto do Padrao Restaurante,0.087935,east,Rua Senhora dos Remedios,Lousa,Portugal,Rua Senhora dos Remedios Padrão Industrial Zon...,3200-193,restaurant
2,6437542.0,Restaurante Fornos de Sao Domingos,0.543903,north,"Rua do Apeadeiro, Fornos",Coimbra,Portugal,"Rua do Apeadeiro, Fornos, Coimbra 3020-927 Por...",3020-927,restaurant
3,12311563.0,Restaurante Arco-Iris Pizzaria,0.604112,northwest,"Rua da Fontinha, n0 5, Corujeira",Coimbra,Portugal,"Rua da Fontinha, n0 5, Corujeira Corujeira Sao...",3045-069,restaurant
4,25333498.0,Padaria Central de Sao Martinho,0.681910,northeast,Rua eng Julio Araujo Vieira 25a,Coimbra,Portugal,"Rua eng Julio Araujo Vieira 25a 25a, Coimbra 3...",3040-300,restaurant
5,14782747.0,Lua Nova,0.757708,northeast,"Rua padre Julio Marques, Lt2, Rc",Coimbra,Portugal,"Rua padre Julio Marques, Lt2, Rc, Coimbra 3045...",3045-108,restaurant
...,...,...,...,...,...,...,...,...,...,...
5386,1899880.0,Residencial Dora,0.438922,northwest,Largo Senhora-a-Branca 92 94,Braga,Portugal,"Largo Senhora-a-Branca 92 94, Braga 4710-926 P...",4710-926,hotel
5387,613552.0,Hotel Bracara Augusta,0.588442,west,Avenida Central no 134,Braga,Portugal,"Avenida Central no 134, Braga 4710-229 Portugal",4710-229,hotel
5388,633325.0,Hotel Residencial Avenida,0.736528,west,Avenida Central,Braga,Portugal,"Avenida Central, Braga 4710-228 Portugal",4710-228,hotel
5389,324282.0,Hotel Senhora-a-Branca,0.445498,northwest,Largo Sra. A. Branca 58,Braga,Portugal,"Largo Sra. A. Branca 58, Braga 4710-443 Portugal",4710-443,hotel


In [87]:
#portugal_dates_complete.isnull().sum()
#portugal_dates_complete.duplicated()

In [93]:
coordinates = pd.read_csv('finalcoordinates_forAPI.csv')
coordinates.head()

,postal_code,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal,nome_concelho,nome_distrito,LatLong
0,3810-081,1,5,6684,Aveiro,3810,81,AVEIRO,Aveiro,Aveiro,40.62887582899364%2C-8.65228518482195
1,3810-241,1,5,6684,Aveiro,3810,241,AVEIRO,Aveiro,Aveiro,40.61406567266208%2C-8.622342018296207
2,3810-034,1,5,6684,Aveiro,3810,34,AVEIRO,Aveiro,Aveiro,40.63989803988801%2C-8.623689068784762
3,3810-242,1,5,6684,Aveiro,3810,242,AVEIRO,Aveiro,Aveiro,40.638956674813514%2C-8.614828028702632
4,3810-245,1,5,6684,Aveiro,3810,245,AVEIRO,Aveiro,Aveiro,40.616296089045086%2C-8.621956502189505


In [96]:
coordinates.drop('cod_distrito', axis=1, inplace=True)

In [99]:
coordinates.drop(['cod_concelho', "cod_localidade","nome_localidade", "num_cod_postal", "ext_cod_postal", "desig_postal", "nome_concelho", "nome_distrito"], axis=1, inplace=True)

In [101]:
final_portugal_dates = pd.merge(coordinates, portugal_dates_complete, on='postal_code')
final_portugal_dates

,postal_code,LatLong,location_id,name,distance,bearing,street,city,country,complete_address,type
0,3810-082,40.64045267512721%2C-8.648788675434677,17593319.0,Cereal World Aveiro,0.059939,northwest,Avenida 5 de Outubro 18,Aveiro,Portugal,"Avenida 5 de Outubro 18, Aveiro 3810-082 Portugal",restaurant
1,3810-082,40.64045267512721%2C-8.648788675434677,23487448.0,Bema,0.063202,west,Avenida 5 de Outubro 41,Aveiro,Portugal,"Avenida 5 de Outubro 41, Aveiro 3810-082 Portugal",restaurant
2,3810-082,40.64045267512721%2C-8.648788675434677,17593319.0,Cereal World Aveiro,0.091396,northwest,Avenida 5 de Outubro 18,Aveiro,Portugal,"Avenida 5 de Outubro 18, Aveiro 3810-082 Portugal",restaurant
3,3810-082,40.64045267512721%2C-8.648788675434677,23487448.0,Bema,0.074411,west,Avenida 5 de Outubro 41,Aveiro,Portugal,"Avenida 5 de Outubro 41, Aveiro 3810-082 Portugal",restaurant
4,3810-082,40.64045267512721%2C-8.648788675434677,23487448.0,Bema,0.111488,north,Avenida 5 de Outubro 41,Aveiro,Portugal,"Avenida 5 de Outubro 41, Aveiro 3810-082 Portugal",restaurant
...,...,...,...,...,...,...,...,...,...,...,...
12052,7005-638,38.58887763863615%2C-7.909512996640514,21184546.0,El Loro Tacos,0.575836,east,Rua do Viveiro 5,Evora,Portugal,"Rua do Viveiro 5, Evora 7005-638 Portugal",restaurant
12053,7005-638,38.58887763863615%2C-7.909512996640514,21184546.0,El Loro Tacos,0.919816,north,Rua do Viveiro 5,Evora,Portugal,"Rua do Viveiro 5, Evora 7005-638 Portugal",restaurant
12054,7005-638,38.58887763863615%2C-7.909512996640514,21184546.0,El Loro Tacos,1.228271,southeast,Rua do Viveiro 5,Evora,Portugal,"Rua do Viveiro 5, Evora 7005-638 Portugal",restaurant
12055,7005-638,38.58887763863615%2C-7.909512996640514,21184546.0,El Loro Tacos,0.202195,northwest,Rua do Viveiro 5,Evora,Portugal,"Rua do Viveiro 5, Evora 7005-638 Portugal",restaurant


In [111]:
final_portugal_dates["name"].value_counts()

The Noble House               168
Mercure Braga Centro           85
Hotel Ibn-Arrik                85
Pensao Residencial Antunes     84
Despertar Saudade AL           84
                             ... 
Golfinho Bar                    1
A Cozinha da Maria              1
Pepe Kebab                      1
DOPPO Gelataria                 1
Toca do Bacalhau                1
Name: name, Length: 1251, dtype: int64

In [114]:
final_portugal_dates = final_portugal_dates.drop_duplicates(subset=['name'])


In [105]:
#final_portugal_dates.isnull().sum()
#final_portugal_dates.duplicated()
len(final_portugal_dates)

12057

In [127]:
final_portugal_dates.to_csv('tripadvisornearbysearch.csv', index=False)

In [118]:
final_portugal_dates = final_portugal_dates[final_portugal_dates['name'] != 'Primark']

In [ ]:
final_portugal_dates['name'].str.capitalize()

In [126]:
final_portugal_dates = final_portugal_dates['type'].str.capitalize()

KeyError: 'type'

In [125]:
final_portugal_dates.head()

0     Restaurant
1     Restaurant
8     Restaurant
19    Restaurant
23    Attraction
Name: type, dtype: object